<a href="https://colab.research.google.com/github/hongjae6/ML_DL/blob/main/tensorflow_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q tensorflow-recommenders

     |████████████████████████████████| 85 kB 2.8 MB/s 
     |████████████████████████████████| 462 kB 35.0 MB/s 


In [3]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import pandas as pd
import tensorflow as tf

import tensorflow_recommenders as tfrs

In [ ]:
seed_number = 42

tf.random.set_seed(seed_number)

In [ ]:
# data
data = None
shuffled_data = data.shuffle(100_000, seed = seed_number, reshuffle_each_iteration = False)

train = shuffled_data.take(80_000)
test = shuffled_data.skip(80_000).take(20.000)

In [ ]:
class Model(tf.keras.Model):
  def __init__(self):
    super().__init__()
    embedding_dimension = None #embedding dimension

    self.user_embeddings = tf.keras.Sequential([
      # ...
    ])

    self.data_embeddings = tf.keras.Sequential([
      # ...
    ])

    self.ratings = tf.keras.Sequential([
      tf.keras.layers.Dense(..., activation = 'relu'), # ... -> input
      # models
      tf.keras.layers.Dense(1)
      # result
    ])

  def call(self, inputs):
    user_id, title = inputs
    user_embedding = self.user_embeddings(user_id)
    data_embedding = self.data_embeddings(title)

    return self.ratings(tf.concat([user_embedding, data_embedding], axis = 1))

In [ ]:
id, data = None 
# id: user id 
# data: data
Model()(([id], [data]))

In [ ]:
class TFRSModel(tfrs.models.Model):
  def __init__(self):
    super().__init__()
    self.ranking_model: tf.keras.Model = Model()
    self.tesk: tf.keras.layers.Layer = tfrs.tasks.Ranking(
        loss = tf.keras.losses.MeanSquaredError(),
        metrics = [tf.keras.metrics.RootMeanSquaredError()]
    )

  def call(self, features: Dict[str, tf.Tensor]) -> tf.Tensor:
    return self.ranking_model(
        (features['user_id'], features['title'])
    )

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    labels = features.pop('rating')

    rating_predictions = self(features)

    return self.task(labels = labels, predictions = rating_predictions)

In [ ]:
model = TFRSModel()
model.compile(optimizer = tf.keras.optimizers.Adagrad(leraning_rate = 0.1))

In [ ]:
# batch size
batch_size = None

cached_train = train.shuffle(100_000).batch(batch_size).cache()
cached_test = test.batch(batch_size).cache()

In [ ]:
# epochs
epochs = None
model.fit(cached_train, epochs = epochs)

In [ ]:
model.evaluate(cached_test, return_dict=True)

In [ ]:
tf.saved_model.save(model, "export")